### 2000~2025的美联储利率

In [60]:
import requests
import pandas as pd
from datetime import datetime

start_time = '2018-02-01'
end_time = '2025-02-01'
start_date = datetime(2018,2,1)
end_date = datetime(2025,2,1)
## https://fredaccount.stlouisfed.org/apikeys
api_key = '2799e9ff56082e330f1d816ae93a1f87'
series_id = 'EFFR'
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    observations = data['observations']
    
    df = pd.DataFrame(observations)
    
    df = df[['date', 'value']]
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    ##df.dropna(inplace=True)
    df = df[df['date']>=start_time].reset_index(drop=True)
    df['date'] = pd.to_datetime(df['date'])

    full_date_range = pd.date_range(start=start_time, end=end_time, freq='D')
    df_full = pd.DataFrame({'date': full_date_range})
    df = pd.merge(df_full, df, on='date', how='left')
    ## 补充一些缺失的日期，使用前后均值插入
    df['value'] = df['value'].interpolate(method='linear', limit_direction='both')

    df.to_csv('fed_funds_rate.csv', index=False)
    print("数据已保存为 fed_funds_rate.csv")
    print(df.head())
    print("有所有的对应时间的数据吗？：", (len(df) == (end_date-start_date).days+1))
    print(df.info())
else:
    print("请求失败，状态码:", response.status_code)

数据已保存为 fed_funds_rate.csv
        date  value
0 2018-02-01   1.42
1 2018-02-02   1.42
2 2018-02-03   1.42
3 2018-02-04   1.42
4 2018-02-05   1.42
有所有的对应时间的数据吗？： True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2558 non-null   datetime64[ns]
 1   value   2558 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.1 KB
None


### 处理具体加息降息的时间点

In [45]:
import requests
import pandas as pd

series_id1 = 'DFEDTARU'  # 选择目标利率上限，或换成 DFEDTARL / DFEDTAR
series_id2 = 'DFEDTARL'

# 请求数据
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id1}&api_key={api_key}&file_type=json'
response = requests.get(url)
data = response.json()
observations = data['observations']
df_upper = pd.DataFrame(observations)

url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id2}&api_key={api_key}&file_type=json'
response = requests.get(url)
data = response.json()
observations = data['observations']
df_lower = pd.DataFrame(observations)

df_upper['date'] = pd.to_datetime(df_upper['date'])
df_lower['date'] = pd.to_datetime(df_lower['date'])
df_upper.rename(columns={'value': 'upper'}, inplace=True)
df_lower.rename(columns={'value': 'lower'}, inplace=True)
df = pd.merge(df_lower, df_upper, on='date', how='inner')
df = df[['date','lower','upper']]

df['upper'] = pd.to_numeric(df['upper'], errors='coerce')
df['lower'] = pd.to_numeric(df['lower'], errors='coerce')
df.dropna(inplace=True)

df['change'] = df['upper'] - df['upper'].shift(1)
df['event'] = df['change'].apply(lambda x: '加息' if x > 0 else ('降息' if x < 0 else '无变化'))

rate_change_df = df[df['event'] != '无变化'].copy()
rate_change_df = rate_change_df[rate_change_df['date']>start_date].reset_index(drop=True)
print(rate_change_df[['date', 'lower', 'upper', 'change', 'event']].head())

rate_change_df.to_csv('fed_rate_change_history.csv', index=False)
print("加息/降息历史数据已保存为 fed_rate_change_history.csv")


        date  lower  upper  change event
0 2018-03-22   1.50   1.75    0.25    加息
1 2018-06-14   1.75   2.00    0.25    加息
2 2018-09-27   2.00   2.25    0.25    加息
3 2018-12-20   2.25   2.50    0.25    加息
4 2019-08-01   2.00   2.25   -0.25    降息
加息/降息历史数据已保存为 fed_rate_change_history.csv


### 加密市场的恐慌贪婪指数

In [64]:
# https://alternative.me/crypto/fear-and-greed-index/#google_vignette
from datetime import datetime

days = (datetime.today() - start_date).days

url = f'https://api.alternative.me/fng/?limit={days-1}'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data['data'])

df['date'] = pd.to_datetime(df['timestamp'], unit='s')

full_date_range = pd.date_range(start=start_time, end=end_time, freq='D')
df_full = pd.DataFrame({'date': full_date_range})
df = pd.merge(df_full, df, on='date', how='left')

df = df.sort_values('date', ascending=True).reset_index(drop=True)
df = df[['date','value','value_classification']]
df['value'] = df['value'].fillna(method='ffill')
df['value_classification'] = df['value_classification'].fillna(method='ffill')

print(df.head())
df.to_csv('fear_greed_history_sorted.csv', index=False)
print("已保存为 fear_greed_history_sorted.csv")
print("有所有的对应时间的数据吗？：", (len(df) == (end_date-start_date).days+1))
df.info()

        date value value_classification
0 2018-02-01    30                 Fear
1 2018-02-02    15         Extreme Fear
2 2018-02-03    40                 Fear
3 2018-02-04    24         Extreme Fear
4 2018-02-05    11         Extreme Fear
已保存为 fear_greed_history_sorted.csv
有所有的对应时间的数据吗？： True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  2558 non-null   datetime64[ns]
 1   value                 2558 non-null   object        
 2   value_classification  2558 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 60.1+ KB


C:\Users\looik\AppData\Local\Temp\ipykernel_24476\3432250480.py:11: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='s')
C:\Users\looik\AppData\Local\Temp\ipykernel_24476\3432250480.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['value'] = df['value'].fillna(method='ffill')
C:\Users\looik\AppData\Local\Temp\ipykernel_24476\3432250480.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['value_classification'] = df['value_classification'].fillna(method='ffill')


### 获取ETH和ETH的历史价格

In [ ]:
# TokenInsight
from sklearn.linear_model import LinearRegression
import numpy as np
def getTokenPrice(name:str):
    api_key_tokeninsight = "e401311c82004971b85481f6926bccc2"

    url = f"https://api.tokeninsight.com/api/v1/history/coins/{name}?interval=day&length=-1"

    headers = {
        "accept": "application/json",
        "TI_API_KEY": api_key_tokeninsight
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    data = data['data']
    file_name = data['name'] + "_price.csv"

    df = pd.DataFrame(data["market_chart"])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

    full_date_range = pd.date_range(start=start_time, end=end_time, freq='D')
    df_full = pd.DataFrame({'date': full_date_range})
    df = pd.merge(df_full, df, on='date', how='left')

    df = df.sort_values('date', ascending=True).reset_index(drop=True)
    df = df[["date","price","market_cap","vol_spot_24h"]]

    ## 拟合线性回归模型，有一些市值缺失了，可以使用线性回归模型找回缺失值（Price和市值的关系）
    train_df = df.dropna(subset=['market_cap'])
    X_train = train_df[['price', 'vol_spot_24h']]
    Y_train = train_df['market_cap']
    model = LinearRegression()
    model.fit(X_train,Y_train)
    missing_row = df[df['market_cap'].isnull()]
    predicted_market_cap = model.predict(missing_row[['price','vol_spot_24h']])
    df.loc[df['market_cap'].isnull(), 'market_cap'] = predicted_market_cap

    print(df.info())
    df.to_csv(file_name, index=False)
    return df


ETH_price = getTokenPrice(name="bitcoin")
ETH_price = getTokenPrice(name="ethereum")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          2558 non-null   datetime64[ns]
 1   price         2558 non-null   float64       
 2   market_cap    2558 non-null   float64       
 3   vol_spot_24h  2558 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 80.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          2558 non-null   datetime64[ns]
 1   price         2558 non-null   float64       
 2   market_cap    2558 non-null   float64       
 3   vol_spot_24h  2558 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 80.1 KB
None


### 处理ETH、ETH和其他加密货币的市值

In [ ]:
# TokenInsight
df = pd.read_csv("./unprocessedFile/top-coins-day-data-all-tokeninsight-dashboard.csv")
df = df.dropna(axis=0).reset_index(drop=True)

df.rename(columns={'Date':'date'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])

full_date_range = pd.date_range(start=start_time, end=end_time, freq='D')
df_full = pd.DataFrame({'date': full_date_range})
df = pd.merge(df_full, df, on='date', how='left')

a = ETH_price['market_cap'].round() == df['BTC Market Cap'].round()
print(f"和上面获得的BTC市值数据相同吗:{a.mean()>=0.99}")
if a.mean()>=0.99:
    df['BTC Market Cap'] = BTC_price['market_cap']

b = ETH_price['market_cap'].round() == df['ETH Market Cap'].round()
print(f"和上面获得的ETH市值数据相同吗:{b.mean()>=0.99}" )
if b.mean()>=0.99:
    df['ETH Market Cap'] = ETH_price['market_cap']

# 缺失部分数据
train_df = df.dropna(axis=0)
X_train = train_df[['BTC Market Cap', 'ETH Market Cap']]
Y_train = train_df[['Top 10 (Without BTC & ETH)','Others']]

model = LinearRegression()
model.fit(X_train,Y_train)
missing_row = df[df['Others'].isnull()]
predicted = model.predict(missing_row[['BTC Market Cap', 'ETH Market Cap']])
df.loc[df['Others'].isnull(), ['Top 10 (Without BTC & ETH)','Others']] = predicted

df['All'] = df['BTC Market Cap'] + df['ETH Market Cap'] + df['Top 10 (Without BTC & ETH)'] + df['Others']

df.to_csv("marketCap.csv", index=False)
df.head()
df.info()

和上面获得的BTC市值数据相同吗:True
和上面获得的ETH市值数据相同吗:True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        2558 non-null   datetime64[ns]
 1   BTC Market Cap              2558 non-null   float64       
 2   ETH Market Cap              2558 non-null   float64       
 3   Top 10 (Without BTC & ETH)  2558 non-null   float64       
 4   Others                      2558 non-null   float64       
 5   All                         2558 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 120.0 KB


In [ ]:
## 已经忘记从哪里爬了，可以不拿这个值，目前只有成交量有点意义，但市值并不完整
import json
with open('./unprocessedFile/TotalCryptoMarketcap.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

dfs = []

for indicator in data['data']:
    raw_data = json.loads(indicator['data'])
    df = pd.DataFrame(raw_data, columns=['timestamp', indicator['name']])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.normalize()
    full_date_range = pd.date_range(start=start_time, end=end_time, freq='D')
    df_full = pd.DataFrame({'timestamp': full_date_range})
    df = pd.merge(df_full, df, on='timestamp', how='left')

    df[indicator['name']] = pd.to_numeric(df[indicator['name']], errors='coerce')
    df.set_index('timestamp', inplace=True)
    dfs.append(df)

df_result = pd.concat(dfs, axis=1).reset_index(drop=False)
## df_result = df_result.fillna(0)
df_result.columns = ['date','加密货币总市值','加密货币24h总成交量','山寨币总市值','山寨币24h总成交量']
df_result['BTC总市值'] = df_result['加密货币总市值'] - df_result['山寨币总市值']
df_result['BTC 24h总交易量'] = df_result['加密货币24h总成交量'] - df_result['山寨币24h总成交量']

df_result.to_csv("marketCap2.csv")
df_result.head()
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2558 non-null   datetime64[ns]
 1   加密货币总市值      2556 non-null   float64       
 2   加密货币24h总成交量  2556 non-null   float64       
 3   山寨币总市值       2088 non-null   float64       
 4   山寨币24h总成交量   2088 non-null   float64       
 5   BTC总市值       2087 non-null   float64       
 6   BTC 24h总交易量  2087 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 140.0 KB


### 处理活跃地址数据

In [ ]:
# Token Terminal
# 2018-06开始
df = pd.read_csv("./unprocessedFile/ActiveAddresses(daily).csv")

## Bitcoin active address
df_bitcoin = df[['Date','Bitcoin']].copy()
df_bitcoin

## Ethereum active address
df_Ethereum = df[['Date','Ethereum']].copy()
df_Ethereum = df_Ethereum.dropna()

## Top 10(Without BTC & ETH)
df_others = df.drop(columns=['Bitcoin','Ethereum'])
df_others = df_others.fillna(0)
df_others['Others'] = df_others[df_others.columns[1:]].sum(axis=1)
df_others = df_others[df_others['Others']!=0]
df_others = df_others[['Date','Others']]

df = df_bitcoin.merge(df_Ethereum, on='Date', how='outer').merge(df_others, on='Date', how='outer')
df = df.dropna(axis=0).reset_index(drop=True)
df.to_csv("active_address.csv",index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2452 non-null   object 
 1   Bitcoin   2452 non-null   float64
 2   Ethereum  2452 non-null   float64
 3   Others    2452 non-null   float64
dtypes: float64(3), object(1)
memory usage: 76.8+ KB


### ETF数据（BTC&ETH 美国ETF现货）

In [ ]:
# https://coinank.com/zh/etf/EthEtf

with open('./unprocessedFile/cryptoETF.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

ETF_BTC_Trading_Date = datetime(2024,1,11)

df_btc = pd.DataFrame(data["btc_data"])
df_btc = df_btc.drop(columns=['totalNav','price'])
df_btc['ts'] = pd.to_datetime(df_btc['ts'],unit='ms')
df_btc.columns = ['date','BTC_ETF_netflow']
df_btc['BTC_ETF_netflow'] = df_btc['BTC_ETF_netflow'].astype('float')

df_eth = pd.DataFrame(data['eth_data'])
df_eth = df_eth.drop(columns=['list','change'])
df_eth['date'] = pd.to_datetime(df_eth['date'],unit='ms')
df_eth.columns = ['date','ETH_ETF_netflow']

df_btc_volume = pd.DataFrame(data['btc_volume_data'])
df_btc_volume = df_btc_volume.drop(columns=['baseCoin','details'])
df_btc_volume['ts'] = pd.to_datetime(df_btc_volume['ts'],unit='ms')
df_btc_volume.columns = ['date','BTC_ETF_Volume/24h']

df_eth_volume = pd.DataFrame(data['eth_volume_data'])
df_eth_volume = df_eth_volume.drop(columns=['baseCoin','details'])
df_eth_volume['ts'] = pd.to_datetime(df_eth_volume['ts'],unit='ms')
df_eth_volume.columns = ['date','ETH_ETF_Volume/24h']

df = pd.merge(df_btc, df_eth, on='date', how='outer') 
df = pd.merge(df, df_btc_volume, on='date', how='outer')
df = pd.merge(df, df_eth_volume, on='date', how='outer')

df = df[df['BTC_ETF_netflow']!=0]

full_date_range = pd.date_range(start=ETF_BTC_Trading_Date, end=end_time, freq='D')
df_full = pd.DataFrame({'date': full_date_range})
df = pd.merge(df_full, df, on='date', how='left')
df = df.fillna(0)

df.to_csv("crypto_ETF_Data.csv")

print(df)


          date  BTC_ETF_netflow
0   2025-03-13    -1.432981e+08
1   2025-03-12     1.333261e+07
2   2025-03-11    -3.710029e+08
3   2025-03-10    -3.694227e+08
4   2025-03-07    -4.092131e+08
..         ...              ...
287 2024-01-18     1.033657e+07
288 2024-01-17     5.175031e+08
289 2024-01-16    -8.058183e+07
290 2024-01-12     1.909738e+08
291 2024-01-11     6.280696e+08

[292 rows x 2 columns]
          date  BTC_ETF_netflow  ETH_ETF_netflow  BTC_ETF_Volume/24h  \
0   2024-01-11     6.280696e+08     0.000000e+00        4.534381e+09   
1   2024-01-12     1.909738e+08     0.000000e+00        3.155793e+09   
2   2024-01-13     0.000000e+00     0.000000e+00        0.000000e+00   
3   2024-01-14     0.000000e+00     0.000000e+00        0.000000e+00   
4   2024-01-15     0.000000e+00     0.000000e+00        0.000000e+00   
..         ...              ...              ...                 ...   
383 2025-01-28     1.843791e+07     0.000000e+00        2.439091e+09   
384 2025-01-29  

### 各交易所的BTC资金费率

In [147]:
# BINANCE（Website）
df_binance = pd.read_csv("./unprocessedFile/Binance Funding Rate History_BTCUSDT.csv")
df_binance = df_binance[['Time','Funding Rate']]
df_binance = df_binance.sort_values('Time', ascending=True).reset_index(drop=True)
df_binance['Time'] = pd.to_datetime(df_binance['Time']).dt.normalize()
df_binance['Funding Rate'] = df_binance['Funding Rate'].str.rstrip('%').astype(float) / 100  # 转成小数形式
df_binance.columns = ['Time','Binance Funding Rate']
df_binance

,Time,Binance Funding Rate
0,2019-09-10,0.000100
1,2019-09-11,0.000100
2,2019-09-11,0.000100
3,2019-09-11,0.000100
4,2019-09-12,0.000100
...,...,...
6028,2025-03-12,0.000087
6029,2025-03-12,0.000006
6030,2025-03-12,0.000058
6031,2025-03-13,0.000040


In [157]:
# BYBIT（Website + library）
from pybit.unified_trading import HTTP
import time
df_bybit = pd.read_csv("./unprocessedFile/Bybit funding_rate_history_BTCUSDT.csv")
df_bybit = df_bybit[['Time','Funding Rate']]
df_bybit['Time'] = pd.to_datetime(df_bybit['Time']) 
df_bybit['Funding Rate'] = df_bybit['Funding Rate'].astype(float)
df_bybit = df_bybit.sort_values('Time', ascending=True).reset_index(drop=True)

start_date = int(datetime(2025,2,25).timestamp() * 1000)
end_date = int(datetime.today().timestamp() * 1000)
limit = 200
session = HTTP()
current_start = start_date
data_bybit = []

while current_start < end_date:
    response = session.get_funding_rate_history(
        category="linear",
        symbol="BTCPERP",
        startTime = current_start,
        endTime = end_date,
        limit = limit
    )
    result = response.get('result', {}).get('list', [])
    if not result:
        break

    data_bybit.extend(result) 
    last_time = int(result[0]['fundingRateTimestamp'])
    current_start = last_time + 1 

    time.sleep(0.2)

df_bybit2 = pd.DataFrame(data_bybit)
df_bybit2['fundingRateTimestamp'] = pd.to_datetime(df_bybit2['fundingRateTimestamp'],unit='ms').reset_index(drop=True)
df_bybit2 = df_bybit2.drop(columns='symbol')
df_bybit2 = df_bybit2[['fundingRateTimestamp','fundingRate']]
df_bybit2['fundingRate'] = df_bybit2['fundingRate'].astype(float)
df_bybit2.columns = ['Time', 'Funding Rate']
df_bybit2 = df_bybit2.reset_index(drop=True) 

df_bybit = pd.concat([df_bybit, df_bybit2], axis=0, ignore_index=True)
df_bybit = df_bybit.sort_values('Time', ascending=True).reset_index(drop=True)
df_bybit.columns = ['Time', 'Bybit Funding Rate']
df_bybit['Time'] = pd.to_datetime(df_bybit['Time']).dt.normalize()
df_bybit

C:\Users\looik\AppData\Local\Temp\ipykernel_24476\113574404.py:36: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_bybit2['fundingRateTimestamp'] = pd.to_datetime(df_bybit2['fundingRateTimestamp'],unit='ms').reset_index(drop=True)


,Time,Bybit Funding Rate
0,2020-03-25,0.000100
1,2020-03-26,0.000100
2,2020-03-26,0.000100
3,2020-03-26,0.000100
4,2020-03-27,0.000100
...,...,...
5442,2025-03-13,0.000079
5443,2025-03-13,0.000100
5444,2025-03-13,0.000070
5445,2025-03-14,0.000081


In [164]:
# BITGET（API）
dfs = []
for i in range(1,43):
    url = "https://api.bitget.com/api/v2/mix/market/history-fund-rate"
    params = {
        "symbol":"BTCUSDT",
        "productType": "usdt-futures",
        "pageSize":100,
        "pageNo":i
    }
    response = requests.get(url, params=params)
    data_bitget = response.json()
    dfs.append(data_bitget['data'])
    
dfs = [item for i in dfs for item in i]
df_bitget = pd.DataFrame(dfs)
df_bitget['fundingTime'] = pd.to_datetime(df_bitget['fundingTime'], unit='ms')
df_bitget = df_bitget.sort_values('fundingTime', ascending=True).reset_index(drop=True)
df_bitget = df_bitget.drop(columns=['symbol'])
df_bitget['fundingRate'] = df_bitget['fundingRate'].astype(float)
df_bitget = df_bitget[['fundingTime','fundingRate']]
df_bitget.columns = ['Time', 'Bitget Funding Rate']
df_bitget['Time'] = pd.to_datetime(df_bitget['Time']).dt.normalize()
df_bitget


C:\Users\looik\AppData\Local\Temp\ipykernel_24476\4248307681.py:17: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_bitget['fundingTime'] = pd.to_datetime(df_bitget['fundingTime'], unit='ms')


,Time,Bitget Funding Rate
0,2021-05-18,0.000000
1,2021-05-18,0.000000
2,2021-05-19,0.000500
3,2021-05-19,-0.000310
4,2021-05-19,0.000500
...,...,...
4183,2025-03-13,0.000011
4184,2025-03-13,0.000027
4185,2025-03-13,0.000046
4186,2025-03-14,0.000072


In [197]:
df_binance = df_binance.groupby('Time', as_index=False)['Binance Funding Rate'].mean()
df_bybit = df_bybit.groupby('Time', as_index=False)['Bybit Funding Rate'].mean()
df_bitget = df_bitget.groupby('Time', as_index=False)['Bitget Funding Rate'].mean()

df_funding_rate = pd.merge(df_binance, df_bybit, on='Time', how='left')
df_funding_rate = pd.merge(df_funding_rate, df_bitget, on='Time', how='left')
cols = ['Binance Funding Rate', 'Bybit Funding Rate', 'Bitget Funding Rate']
df_funding_rate['Mean Funding Rate'] = df_funding_rate[cols].mean(axis=1,skipna=True)
df_funding_rate.to_csv('funding_rate.csv')
df_funding_rate

,Time,Binance Funding Rate,Bybit Funding Rate,Bitget Funding Rate,Mean Funding Rate
0,2019-09-10,0.000100,NaN,NaN,0.000100
1,2019-09-11,0.000100,NaN,NaN,0.000100
2,2019-09-12,0.000100,NaN,NaN,0.000100
3,2019-09-13,0.000100,NaN,NaN,0.000100
4,2019-09-14,0.000100,NaN,NaN,0.000100
...,...,...,...,...,...
2007,2025-03-09,0.000005,0.000025,0.000047,0.000026
2008,2025-03-10,0.000051,-0.000007,0.000043,0.000029
2009,2025-03-11,0.000040,0.000098,0.000065,0.000068
2010,2025-03-12,0.000050,0.000076,0.000040,0.000055


### BTC、ETH和加密货币的净流入流出量

In [170]:
# 一周为单位（还在找着有没有一天的）
# IntoTheBlock
with open('./unprocessedFile/crypto_exchange_netflows.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def crypto_netflow(token_name:str):

    df = pd.DataFrame(data[token_name]['metric'])
    df = df[[0,1]]
    df[1] = df[1].apply(lambda x: x[0] if isinstance(x, list) else None)
    df = df.dropna(axis=0)
    df[0] = pd.to_datetime(df[0], unit='ms').reset_index(drop=True)
    df.columns = ['time', 'netflow']
    return df

df_ETH_Netflow = crypto_netflow("ETH")
df_BTC_Netflow = crypto_netflow("BTC")

df_crypto_Netflow = pd.merge(df_BTC_Netflow, df_ETH_Netflow, on='time', how='outer')
df_crypto_Netflow.columns = ['time(weeks)','BTC_Netflow','ETH_Netflow']

df_crypto_Netflow


,time(weeks),BTC_Netflow,ETH_Netflow
0,2011-07-11,1.378733e+01,NaN
1,2011-07-18,-1.160835e+01,NaN
2,2011-07-25,1.333309e+01,NaN
3,2011-08-01,-3.846900e+00,NaN
4,2011-08-08,-1.094747e+01,NaN
...,...,...,...
707,2025-01-27,6.795580e+07,3.044049e+08
708,2025-02-03,1.324378e+09,-7.904834e+08
709,2025-02-10,2.170963e+08,-1.124986e+08
710,2025-02-17,1.012559e+09,-2.644868e+08


### 稳定币发行量和净流入流出量

In [199]:
# USDC, DAI, FDUSD, TUSD
# https://intel.arkm.com/dashboards/view?dashboardID=3c55c261-2e87-43c3-9748-3fbc95603b97

with open('./unprocessedFile/stablecoin_exchange_flow.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def stablecoin_netflow(token_name:str):

    df = pd.DataFrame(data[token_name])
    df = df[(df['inUSD']>0) | (df['outUSD']>0)]
    df = df.sort_values('time', ascending=True).reset_index(drop=True)
    df['netUSD'] = df['inUSD'] - df['outUSD']
    df['netValue'] = df['inValue'] - df['outValue']
    df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)
    df = df[['time','inUSD','outUSD','netUSD','inValue','outValue','netValue']]
    df.rename(columns={'netUSD': f'net_{token_name}(USD)'}, inplace=True)
    return df

df_usdc = stablecoin_netflow('usdc')
df_dai = stablecoin_netflow('dai')
df_fdusd = stablecoin_netflow('fdusd')
df_tusd = stablecoin_netflow('tusd')

df = pd.merge(df_usdc[['time','net_usdc(USD)']], df_dai[['time','net_dai(USD)']], on='time', how='outer')
df = pd.merge(df, df_fdusd[['time','net_fdusd(USD)']], on='time', how='outer')
df = pd.merge(df, df_tusd[['time','net_tusd(USD)']], on='time', how='outer')

full_date_range = pd.date_range(start='2018-10-5', end=end_time, freq='D')
df_full = pd.DataFrame({'time': full_date_range})
df = pd.merge(df_full, df, on='time', how='left')
df['Total Net'] = df.drop(columns=['time']).sum(axis=1,skipna=True)
df.to_csv("stablecoin_netflow.csv")
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312 entries, 0 to 2311
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   time            2312 non-null   datetime64[ns]
 1   net_usdc(USD)   2312 non-null   float64       
 2   net_dai(USD)    1902 non-null   float64       
 3   net_fdusd(USD)  552 non-null    float64       
 4   net_tusd(USD)   2221 non-null   float64       
 5   Total Net       2312 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 108.5 KB


,time,net_usdc(USD),net_dai(USD),net_fdusd(USD),net_tusd(USD),Total Net
0,2018-10-05,3.247538e+06,NaN,NaN,NaN,3.247538e+06
1,2018-10-06,-2.503724e+04,NaN,NaN,NaN,-2.503724e+04
2,2018-10-07,-4.162892e+03,NaN,NaN,NaN,-4.162892e+03
3,2018-10-08,4.632953e+03,NaN,NaN,NaN,4.632953e+03
4,2018-10-09,4.030299e+05,NaN,NaN,NaN,4.030299e+05
...,...,...,...,...,...,...
2307,2025-01-28,-1.497638e+08,-1.088089e+06,-2.353983e+07,28105.441501,-1.743636e+08
2308,2025-01-29,1.512962e+08,2.656269e+06,-2.118591e+06,19495.124602,1.518534e+08
2309,2025-01-30,1.707359e+08,2.949197e+06,3.874627e+05,39956.454841,1.741125e+08
2310,2025-01-31,-5.297730e+07,-7.548476e+05,-1.425597e+06,4347.873433,-5.515339e+07


In [200]:
# All stablecoin's netflow in crypto market
# IntoTheBlock

with open('./unprocessedFile/allStablecoin_exchange_flows.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df_all_stablecoin = pd.DataFrame(data['data']['rows'])
df_all_stablecoin[0] = pd.to_datetime(df_all_stablecoin[0]).dt.tz_localize(None)
df_all_stablecoin.columns = ['time', 'inflow', 'outflow', 'netflow']
df_all_stablecoin = df_all_stablecoin.sort_values('time', ascending=True).reset_index(drop=True)
df_all_stablecoin.set_index('time', inplace=True)
df_all_stablecoin.to_csv("all_stablecoin_netflow.csv")
df_all_stablecoin.info()
df_all_stablecoin

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2000 entries, 2019-09-21 to 2025-03-12
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   inflow   2000 non-null   float64
 1   outflow  2000 non-null   float64
 2   netflow  2000 non-null   float64
dtypes: float64(3)
memory usage: 62.5 KB


,inflow,outflow,netflow
time,,,
2019-09-21,9.285368e+07,9.678754e+07,-3.933857e+06
2019-09-22,1.935425e+08,1.880211e+08,5.521311e+06
2019-09-23,3.890572e+08,3.527504e+08,3.630681e+07
2019-09-24,3.497708e+08,3.120993e+08,3.767146e+07
2019-09-25,2.019344e+08,2.074047e+08,-5.470372e+06
...,...,...,...
2025-03-08,7.036590e+08,7.376716e+08,-3.401259e+07
2025-03-09,1.049148e+09,9.061097e+08,1.430383e+08
2025-03-10,4.769477e+09,4.949395e+09,-1.799181e+08


### SPY, VIX, 美元指数

In [22]:
# https://finance.yahoo.com/quote/%5EGSPC/
# https://finance.yahoo.com/quote/DX-Y.NYB/
# https://finance.yahoo.com/quote/%5EVIX/

from __future__ import annotations

import pandas as pd
import yfinance as yf

ticker_list = [
    '^GSPC',
    'DX-Y.NYB',
    '^VIX'
]

START_DATE="2018-01-01"
END_DATE="2025-03-14"
TIME_INTERVAL= "1D"

def download_data(
    ticker_list: list[str],
    start_date: str,
    end_date: str,
    time_interval: str,
    proxy: str | dict = None,
) -> pd.DataFrame:

    # Download and save the data in a pandas DataFrame
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    data_df = pd.DataFrame()
    for tic in ticker_list:
        temp_df = yf.download(
            tic,
            start=start_date,
            end=end_date,
            interval=time_interval,
            proxy=proxy,
        )
        temp_df.columns = temp_df.columns.droplevel(1)
        temp_df["tic"] = tic
        data_df = pd.concat([data_df, temp_df])

    return data_df

us_stock_data=download_data(ticker_list=ticker_list,start_date=START_DATE,end_date=END_DATE,time_interval=TIME_INTERVAL)
print(us_stock_data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open      Volume  \
Date                                                                         
2018-01-02  2695.810059  2695.889893  2682.360107  2683.729980  3397430000   
2018-01-03  2713.060059  2714.370117  2697.770020  2697.850098  3544030000   
2018-01-04  2723.989990  2729.290039  2719.070068  2719.310059  3697340000   
2018-01-05  2743.149902  2743.449951  2727.919922  2731.330078  3239280000   
2018-01-08  2747.709961  2748.510010  2737.600098  2742.669922  3246160000   
...                 ...          ...          ...          ...         ...   
2025-03-07    23.370001    26.559999    23.090000    24.850000           0   
2025-03-10    27.860001    29.559999    24.680000    24.700001           0   
2025-03-11    26.920000    29.570000    26.180000    27.940001           0   
2025-03-12    24.230000    26.910000    23.889999    26.879999           0   
2025-03-13    25.139999    25.320000    23.459999    24.920000  

In [206]:
df = pd.DataFrame(us_stock_data)
df = df.drop(columns=['High','Low','Open']).reset_index(drop=False)
df_spy = df[df['tic']=='^GSPC'].drop(columns=['tic']).rename(columns={'Close':'SPY_Close','Volume':'SPY_Volume'})
df_vix = df[df['tic']=='^VIX'].drop(columns=['Volume','tic']).rename(columns={'Close':'VIX_Close'})
df_UsIndex = df[df['tic']=='DX-Y.NYB'].drop(columns=['Volume','tic']).rename(columns={'Close':'UsIndex_Close'})

df = pd.merge(df_spy, df_vix, on='Date', how='outer')
df = pd.merge(df, df_UsIndex, on='Date', how='outer')

df.to_csv("spy_vix_usIndex.csv")
df

Price,Date,SPY_Close,SPY_Volume,VIX_Close,UsIndex_Close
0,2018-01-02,2695.810059,3.397430e+09,9.770000,91.849998
1,2018-01-03,2713.060059,3.544030e+09,9.150000,92.160004
2,2018-01-04,2723.989990,3.697340e+09,9.220000,91.849998
3,2018-01-05,2743.149902,3.239280e+09,9.220000,91.949997
4,2018-01-08,2747.709961,3.246160e+09,9.520000,92.330002
...,...,...,...,...,...
1805,2025-03-07,5770.200195,5.705140e+09,23.370001,103.839996
1806,2025-03-10,5614.560059,6.409370e+09,27.860001,103.910004
1807,2025-03-11,5572.069824,6.221240e+09,26.920000,103.440002
1808,2025-03-12,5599.299805,5.219830e+09,24.230000,103.610001


### 相关性检验

4     2019-08-01
5     2019-09-19
6     2019-10-31
7     2020-03-04
8     2020-03-16
20    2024-09-19
21    2024-11-08
22    2024-12-19
Name: date, dtype: object

In [273]:
# 整合数据
""" 
    时间，利率，利率变化，BTC价格变化，ETH价格变化，恐慌贪婪指数（crypto），BTC市值变化，ETH市值变化，
    TOP10市值变化，其他币总市值变化，crypto总市值变化，活跃地址变化，BTC&ETH净流入流出（包括ETF），稳定币净流入流出,
    SPY, VIX, 美元指数
"""

"""increase_fed_rate_date decrease_fed_rate_date"""
# fed_rate_change_history.csv
# date
fed_rate_change = pd.read_csv('fed_rate_change_history.csv')
increase_fed_rate_date = fed_rate_change[fed_rate_change['event'] == '加息']['date']
decrease_fed_rate_date = fed_rate_change[fed_rate_change['event'] == '降息']['date']

"""fed_funds_rate"""
# fed_funds_rate.csv 利率信息
# date | 利率 | 利率变化率
fed_funds_rate = pd.read_csv('fed_funds_rate.csv')
fed_funds_rate.rename(columns={'value':'利率'},inplace=True)
fed_funds_rate['利率变化率'] = fed_funds_rate['利率'].pct_change()
fed_funds_rate.fillna(0,inplace=True)
print(fed_funds_rate)

"""fear_greed_history"""
# fear_greed_history_sorted.csv 恐慌贪婪指数 crypto
# date | Crypto恐慌贪婪指数 | 指数分布 | Crypto恐慌贪婪指数变化率
fear_greed_history = pd.read_csv("fear_greed_history_sorted.csv")
mapping = {
    'Extreme Fear': 0,
    'Fear': 1,
    'Neutral': 2,
    'Greed': 3,
    'Extreme Greed': 4
}
fear_greed_history['value_classification'] = fear_greed_history['value_classification'].map(mapping)
fear_greed_history['value_change'] = fear_greed_history['value'].pct_change()
fear_greed_history.columns = ['date','Crypto恐慌贪婪指数','指数分布','Crypto恐慌贪婪指数变化率']

"""BTC_ETH_price"""
# Bitcoin_price.csv Ethereum_price.csv BTC & ETH 价格变化
# |date |价格(BTC) |市值(BTC) |24H成交量(BTC) |价格变化(BTC) |市值变化(BTC) |24H成交量变化(BTC) |价格(ETH) |市值(ETH) |24H成交量(ETH) |价格变化(ETH) |市值变化(ETH) |24H成交量变化(ETH)
BTC_price = pd.read_csv('Bitcoin_price.csv')
BTC_price.columns = ["date","价格(BTC)","市值(BTC)","24H成交量(BTC)"]
BTC_price['价格变化(BTC)'] = BTC_price['价格(BTC)'].pct_change()
BTC_price['市值变化(BTC)'] = BTC_price['市值(BTC)'].pct_change()
BTC_price['24H成交量变化(BTC)'] = BTC_price['24H成交量(BTC)'].pct_change()

ETH_price = pd.read_csv('Ethereum_price.csv')
ETH_price.columns = ["date","价格(ETH)","市值(ETH)","24H成交量(ETH)"]
ETH_price['价格变化(ETH)'] = ETH_price['价格(ETH)'].pct_change()
ETH_price['市值变化(ETH)'] = ETH_price['市值(ETH)'].pct_change()
ETH_price['24H成交量变化(ETH)'] = ETH_price['24H成交量(ETH)'].pct_change()

BTC_ETH_price = pd.merge(BTC_price,ETH_price,on='date',how='outer')

"""marketcap"""
# marketCap.csv TOP10市值变化，其他币总市值变化，crypto总市值变化
# |date |Top10总市值(不包括BTC&ETH) |其他币总市值 |加密货币总市值 |Top10总市值变化(不包括BTC&ETH) |其他币总市值变化 |加密货币总市值变化
marketcap = pd.read_csv('marketCap.csv')
marketcap = marketcap.drop(columns=['BTC Market Cap','ETH Market Cap'])
marketcap.columns = ['date','Top10总市值(不包括BTC&ETH)','其他币总市值','加密货币总市值']
marketcap['Top10总市值变化(不包括BTC&ETH)'] = marketcap['Top10总市值(不包括BTC&ETH)'].pct_change()
marketcap['其他币总市值变化'] = marketcap['其他币总市值'].pct_change()
marketcap['加密货币总市值变化'] = marketcap['加密货币总市值'].pct_change()

"""active_address"""
# active_address.csv 活跃地址数据
# |date |BTC活跃地址 |ETH活跃地址 |其他活跃地址 |BTC活跃地址变化 |ETH活跃地址变化 |其他活跃地址变化
active_address = pd.read_csv("active_address.csv")
active_address.columns = ['date','BTC活跃地址','ETH活跃地址','其他活跃地址']
active_address['BTC活跃地址变化'] = active_address['BTC活跃地址'].pct_change()
active_address['ETH活跃地址变化'] = active_address['ETH活跃地址'].pct_change()
active_address['其他活跃地址变化'] = active_address['其他活跃地址'].pct_change()

# crypto_ETF_Data.csv BTC&ETH的ETF净流入

"""funding_rate"""
# funding_rate.csv 平均资金费率
# |date |平均资金费率 |平均资金费率变化
funding_rate = pd.read_csv('funding_rate.csv')
funding_rate = funding_rate[['Time', 'Mean Funding Rate']]
funding_rate['Mean Funding Rate Change'] = funding_rate['Mean Funding Rate'].pct_change()
funding_rate.columns = ['date', '平均资金费率', '平均资金费率变化'] 

"""stablecoin_netflow"""
# stablecoin_netflow.csv all_stablecoin_netflow.csv 稳定币净流入流出
# |date |净流入(USDC/DAI/FDUSD/TUSD) |净流入变化(USDC/DAI/FDUSD/TUSD) |总稳定币净流入 |总稳定币净流入变化
stablecoin_netflow = pd.read_csv('stablecoin_netflow.csv')
stablecoin_netflow = stablecoin_netflow[['time','Total Net']]
stablecoin_netflow['Total Net Change'] = stablecoin_netflow['Total Net'].pct_change()

all_stablecoin_netflow = pd.read_csv('all_stablecoin_netflow.csv')
all_stablecoin_netflow = all_stablecoin_netflow[['time','netflow']]
all_stablecoin_netflow['netflow change'] = all_stablecoin_netflow['netflow'].pct_change()

stablecoin_netflow = pd.merge(stablecoin_netflow, all_stablecoin_netflow, on='time',how='left')
stablecoin_netflow.columns = ['date','净流入(USDC/DAI/FDUSD/TUSD)','净流入变化(USDC/DAI/FDUSD/TUSD)','总稳定币净流入','总稳定币净流入变化']

# spy_vix_usIndex.csv  SPY & VIX & UsIndex数据


            date    利率  利率变化率
0     2018-02-01  1.42    0.0
1     2018-02-02  1.42    0.0
2     2018-02-03  1.42    0.0
3     2018-02-04  1.42    0.0
4     2018-02-05  1.42    0.0
...          ...   ...    ...
2553  2025-01-28  4.33    0.0
2554  2025-01-29  4.33    0.0
2555  2025-01-30  4.33    0.0
2556  2025-01-31  4.33    0.0
2557  2025-02-01  4.33    0.0

[2558 rows x 3 columns]


In [289]:
# 所有数据结合
from functools import reduce

dataFrames = [fed_funds_rate, fear_greed_history, BTC_ETH_price, marketcap, active_address, funding_rate, stablecoin_netflow]
df_merged = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dataFrames)
df_merged.dropna(axis=0,inplace=True)
df_merged.set_index('date',inplace=True)

increase_fed_rate_date = increase_fed_rate_date[(increase_fed_rate_date > df_merged.index[0]) & (increase_fed_rate_date < df_merged.index[-1])]
increase_fed_rate_date = pd.to_datetime(increase_fed_rate_date)
decrease_fed_rate_date = decrease_fed_rate_date[(decrease_fed_rate_date > df_merged.index[0]) & (decrease_fed_rate_date < df_merged.index[-1])]
decrease_fed_rate_date = pd.to_datetime(decrease_fed_rate_date)

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1960 entries, 2019-09-22 to 2025-02-01
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   利率                          1960 non-null   float64
 1   利率变化率                       1960 non-null   float64
 2   Crypto恐慌贪婪指数                1960 non-null   float64
 3   指数分布                        1960 non-null   float64
 4   Crypto恐慌贪婪指数变化率             1960 non-null   float64
 5   价格(BTC)                     1960 non-null   float64
 6   市值(BTC)                     1960 non-null   float64
 7   24H成交量(BTC)                 1960 non-null   float64
 8   价格变化(BTC)                   1960 non-null   float64
 9   市值变化(BTC)                   1960 non-null   float64
 10  24H成交量变化(BTC)               1960 non-null   float64
 11  价格(ETH)                     1960 non-null   float64
 12  市值(ETH)                     1960 non-null   float64
 13  24H成交量(ETH)            

In [274]:
# 皮尔森相关系数（Pearson）
"""
    结论：利率或利率变化率对市场没有太大的相关性
"""
corr = df_merged.corr()
corr


,利率,利率变化率,Crypto恐慌贪婪指数,指数分布,Crypto恐慌贪婪指数变化率,价格(BTC),市值(BTC),24H成交量(BTC),价格变化(BTC),市值变化(BTC),...,其他活跃地址,BTC活跃地址变化,ETH活跃地址变化,其他活跃地址变化,平均资金费率,平均资金费率变化,净流入(USDC/DAI/FDUSD/TUSD),净流入变化(USDC/DAI/FDUSD/TUSD),总稳定币净流入,总稳定币净流入变化
利率,1.000000,-0.014604,0.197892,0.227819,-0.028604,0.338566,0.364876,-0.557449,0.001948,0.001689,...,0.635441,-0.008012,0.009926,-0.029614,-0.187890,0.016292,-0.021597,-0.031145,-0.058607,0.021478
利率变化率,-0.014604,1.000000,-0.021565,-0.011099,0.031530,0.013246,0.012166,0.010221,0.039454,0.037608,...,-0.002063,0.009241,-0.021385,0.011089,0.006569,0.008907,0.045822,0.002598,-0.008309,-0.000516
Crypto恐慌贪婪指数,0.197892,-0.021565,1.000000,0.966557,0.050863,0.426790,0.425408,0.061325,0.143789,0.145591,...,0.237373,-0.004912,-0.022852,0.010198,0.496431,0.026842,-0.016321,-0.017865,-0.096594,0.008454
指数分布,0.227819,-0.011099,0.966557,1.000000,0.046507,0.432984,0.432231,0.062732,0.136078,0.138291,...,0.257800,0.001358,-0.012649,0.009442,0.467858,0.032233,-0.016265,-0.018669,-0.095693,0.014699
Crypto恐慌贪婪指数变化率,-0.028604,0.031530,0.050863,0.046507,1.000000,-0.003270,-0.004250,-0.005504,0.437134,0.442653,...,-0.008378,0.016574,0.022622,-0.010235,-0.034324,0.011888,-0.010910,-0.006272,-0.002763,-0.003159
价格(BTC),0.338566,0.013246,0.426790,0.432984,-0.003270,1.000000,0.999317,0.005101,0.027438,0.026995,...,0.799622,0.001587,-0.021294,-0.017643,0.129854,-0.003162,0.017031,-0.024959,-0.031914,0.028878
市值(BTC),0.364876,0.012166,0.425408,0.432231,-0.004250,0.999317,1.000000,-0.015858,0.025810,0.026252,...,0.818198,0.001261,-0.020692,-0.018338,0.116061,-0.002269,0.015674,-0.025732,-0.032958,0.028629
24H成交量(BTC),-0.557449,0.010221,0.061325,0.062732,-0.005504,0.005101,-0.015858,1.000000,0.003463,0.003227,...,-0.279794,0.004686,0.018881,-0.008117,0.430596,-0.038814,0.035986,0.009696,0.030927,-0.002166
价格变化(BTC),0.001948,0.039454,0.143789,0.136078,0.437134,0.027438,0.025810,0.003463,1.000000,0.965055,...,0.004256,0.056239,0.001680,0.074795,0.110457,-0.021171,0.035826,0.003755,0.010900,-0.026971
市值变化(BTC),0.001689,0.037608,0.145591,0.138291,0.442653,0.026995,0.026252,0.003227,0.965055,1.000000,...,0.003652,0.056645,0.016416,0.071940,0.117538,0.020585,0.012041,0.003588,0.006211,-0.026474


In [ ]:
"""
    结论：不相关
"""
from statsmodels.tsa.stattools import grangercausalitytests

target_vars = ['利率', '利率变化率']

# 其他需要测试的列（排除目标列）
other_vars = [col for col in df_merged.columns if col not in target_vars]

# 设定最大滞后阶数
max_lag = 7

# 循环所有组合
for target in target_vars:
    print(f"\n########### Granger Causality Test: {target} ###########\n")
    for col in other_vars:
        print(f"\nTesting if {col} Granger-causes {target}:\n")
        # 注意顺序：[被解释变量, 自变量]
        test_data = df_merged[[target, col]].dropna()
        try:
            result = grangercausalitytests(test_data, maxlag=max_lag, verbose=False)
            # 提取 p-value
            for lag in range(1, max_lag + 1):
                p_value = result[lag][0]['ssr_ftest'][1]
                print(f"Lag {lag}: P-value = {p_value:.4f}")
        except Exception as e:
            print(f"Failed to test {col} -> {target}, error: {e}")

In [291]:
# 存储所有事件窗口的日期
all_event_windows = pd.Series(dtype='datetime64[ns]')
event_dates = fed_rate_change['date']
event_dates = event_dates[(event_dates>df_merged.index[0]) & (event_dates<df_merged.index[-1])]
event_dates = pd.to_datetime(event_dates)

# 遍历每个事件日期，生成前后7天区间
for event_date in event_dates:
    window = pd.date_range(event_date - pd.Timedelta(days=7), event_date + pd.Timedelta(days=7))
    all_event_windows = pd.concat([all_event_windows,pd.Series(window)])

# 去重
all_event_windows = all_event_windows.drop_duplicates().reset_index(drop=True)

df_event = df_merged.copy()
df_event.index = pd.to_datetime(df_event.index)
event_window_data = df_event[df_event.index.isin(all_event_windows)]
event_window_data

,利率,利率变化率,Crypto恐慌贪婪指数,指数分布,Crypto恐慌贪婪指数变化率,价格(BTC),市值(BTC),24H成交量(BTC),价格变化(BTC),市值变化(BTC),...,其他活跃地址,BTC活跃地址变化,ETH活跃地址变化,其他活跃地址变化,平均资金费率,平均资金费率变化,净流入(USDC/DAI/FDUSD/TUSD),净流入变化(USDC/DAI/FDUSD/TUSD),总稳定币净流入,总稳定币净流入变化
date,,,,,,,,,,,,,,,,,,,,,
2019-10-24,1.85,0.000000,20.0,0.0,-0.393939,7468.532532,1.345336e+11,2.337289e+10,-0.071689,-0.070764,...,81582.0,-0.056377,-0.000880,0.427282,0.000100,0.000000,1.806637e+07,-3.412219,-1.611622e+08,1.698031
2019-10-25,1.83,-0.010811,24.0,0.0,0.200000,7433.915346,1.338812e+11,1.882785e+10,-0.004635,-0.004849,...,61895.0,0.139494,0.059517,-0.241315,0.000052,-0.479833,-3.995724e+06,-1.221169,-8.006286e+07,-0.503216
2019-10-26,1.83,0.000000,53.0,2.0,1.208333,8652.030030,1.553916e+11,3.357388e+10,0.163859,0.160668,...,61566.0,-0.032733,0.003746,-0.005315,0.000030,-0.423262,1.372205e+05,-1.034342,-5.280061e+07,-0.340511
2019-10-27,1.83,0.000000,50.0,2.0,-0.056604,9217.236032,1.661183e+11,4.997385e+10,0.065326,0.069031,...,60345.0,-0.266462,-0.149110,-0.019832,0.000198,5.603889,-2.083832e+06,-16.186008,-1.157444e+07,-0.780790
2019-10-28,1.83,0.000000,52.0,2.0,0.040000,9502.020114,1.711916e+11,3.839678e+10,0.030897,0.030540,...,72631.0,0.322559,0.163801,0.203596,0.000336,0.696778,-3.049984e+06,0.463642,3.197945e+08,-28.629367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-22,4.33,0.000000,73.0,3.0,0.000000,95076.631099,1.882503e+12,2.195992e+10,-0.022121,-0.022098,...,14074401.0,0.093742,0.019850,-0.060541,0.000083,0.880149,-1.414643e+07,0.424716,-8.754357e+07,513.280983
2024-12-23,4.33,0.000000,70.0,3.0,-0.041096,94682.388788,1.874739e+12,3.305993e+10,-0.004147,-0.004124,...,15335415.0,0.019181,0.040272,0.089596,0.000080,-0.029977,1.470868e+08,-11.397448,-2.298122e+08,1.625118
2024-12-24,4.33,0.000000,73.0,3.0,0.042857,98636.660202,1.953075e+12,2.262760e+10,0.041764,0.041785,...,14944540.0,-0.029870,-0.014207,-0.025488,0.000092,0.142137,-8.468734e+07,-1.575764,3.043280e+08,-2.324246


In [292]:
event_window_data.corr()

,利率,利率变化率,Crypto恐慌贪婪指数,指数分布,Crypto恐慌贪婪指数变化率,价格(BTC),市值(BTC),24H成交量(BTC),价格变化(BTC),市值变化(BTC),...,其他活跃地址,BTC活跃地址变化,ETH活跃地址变化,其他活跃地址变化,平均资金费率,平均资金费率变化,净流入(USDC/DAI/FDUSD/TUSD),净流入变化(USDC/DAI/FDUSD/TUSD),总稳定币净流入,总稳定币净流入变化
利率,1.000000,-0.076169,0.714854,0.723150,0.014969,0.401875,0.410915,-0.454886,0.113025,0.106878,...,0.542260,0.017066,-0.022214,-0.038705,0.106726,0.018324,-0.026431,-0.058891,-0.144433,0.036388
利率变化率,-0.076169,1.000000,-0.051033,-0.029390,0.096439,0.039451,0.035464,0.026467,0.089708,0.084881,...,-0.020380,0.059388,-0.042300,0.010960,0.074271,0.000641,0.092468,0.009544,-0.035351,-0.020935
Crypto恐慌贪婪指数,0.714854,-0.051033,1.000000,0.966171,0.126396,0.621218,0.625662,-0.258468,0.177830,0.195042,...,0.590062,-0.001076,-0.054237,-0.050979,0.382399,-0.007815,-0.009603,-0.122069,-0.146776,0.061481
指数分布,0.723150,-0.029390,0.966171,1.000000,0.116862,0.572136,0.576996,-0.265600,0.162188,0.186051,...,0.547233,0.020594,-0.039777,-0.037603,0.326275,0.010237,0.004695,-0.124394,-0.123846,0.033397
Crypto恐慌贪婪指数变化率,0.014969,0.096439,0.126396,0.116862,1.000000,0.034277,0.033567,-0.071891,0.325666,0.368664,...,0.025809,-0.118006,-0.040537,-0.043645,0.023531,0.029562,-0.013467,-0.012097,0.039462,0.011714
价格(BTC),0.401875,0.039451,0.621218,0.572136,0.034277,1.000000,0.999827,-0.109225,0.112467,0.117021,...,0.918233,0.013612,-0.052425,-0.047371,0.143015,-0.014657,-0.001740,-0.184556,-0.068422,0.149023
市值(BTC),0.410915,0.035464,0.625662,0.576996,0.033567,0.999827,1.000000,-0.107956,0.108846,0.116796,...,0.922329,0.013693,-0.051630,-0.046580,0.142972,-0.012860,-0.002427,-0.184931,-0.068767,0.149483
24H成交量(BTC),-0.454886,0.026467,-0.258468,-0.265600,-0.071891,-0.109225,-0.107956,1.000000,-0.064179,-0.076248,...,-0.123860,0.030474,0.000033,0.025193,-0.077568,-0.127711,0.033289,-0.063202,0.098014,-0.034555
价格变化(BTC),0.113025,0.089708,0.177830,0.162188,0.325666,0.112467,0.108846,-0.064179,1.000000,0.896143,...,0.115904,0.041350,-0.071248,-0.008884,0.226664,-0.044006,0.023511,-0.030345,-0.112155,-0.034379
市值变化(BTC),0.106878,0.084881,0.195042,0.186051,0.368664,0.117021,0.116796,-0.076248,0.896143,1.000000,...,0.116249,0.032808,-0.023772,-0.009495,0.329937,0.059290,-0.045061,-0.030968,-0.100299,-0.031413
